In [1]:
import os
os.getcwd()

%pip install -e .. -qqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

from utils import save_run, load_run
from config_targets import TARGET_META
from explainer import run_fused_pipeline_for_classes
from selection import build_selection_df_with_aliases

%load_ext autoreload
%autoreload 2

In [3]:
ROOT = Path.cwd().parent

MODEL_PATH         = ROOT / "model" / "resnet_final.keras"
SNOMED_CLASSES_NPY = ROOT / "data" / "snomed_classes.npy"
SEL_DF_CSV         = ROOT / "data" / "ecg_model_pred_data.csv"

ECG_FILENAMES_PATH = ROOT / "data" / "ecg_filenames.npy"
PROBS_PATH         = ROOT / "data" / "ecg_model_probs.npy"
CLASS_NAMES_PATH   = ROOT / "data" / "snomed_classes.npy"
Y_TRUE_PATH        = ROOT / "data" / "ecg_y_true.npy"
ECG_DURATIONS_PATH = ROOT / "data" / "ecg_durations.npy"
ECG_PRED_PATH      = ROOT / "data" / "ecg_xai_sel_meta_p0.85_k5.csv"

model = tf.keras.models.load_model(MODEL_PATH, compile=False)
class_names = np.load(SNOMED_CLASSES_NPY, allow_pickle=True)

In [4]:
from config import DATA_ROOT
from utils import import_key_data
from ecg_predict import batched_predict_all
from selection import build_y_true_from_labels

gender, age, labels, ecg_filenames = import_key_data(DATA_ROOT)

# 1) Build ground-truth multi-hot labels
y_true = build_y_true_from_labels(labels, class_names)

# 2) Predict probabilities
probs = batched_predict_all(
    model,
    ecg_filenames,
    maxlen=5000,
    batch_size=32,
)

# 3) Optional: binary predictions (0/1) at some threshold
pred_threshold = 0.5
y_pred = (probs >= pred_threshold).astype(np.int8)

Y_PRED_PATH      = ROOT / "data" / f"ecg_y_pred_{pred_threshold:.2f}.npy"

# 4) Save everything
np.save(ECG_FILENAMES_PATH, ecg_filenames)
np.save(PROBS_PATH, probs)
np.save(Y_TRUE_PATH, y_true)
np.save(Y_PRED_PATH, y_pred)


Predicting ECGs: 100%|██████████| 413/413 [09:53<00:00,  1.44s/it]


In [5]:
ecg_filenames = np.load(ECG_FILENAMES_PATH, allow_pickle=True)
probs         = np.load(PROBS_PATH)
class_names   = np.load(CLASS_NAMES_PATH, allow_pickle=True)
y_true        = np.load(Y_TRUE_PATH)

sel_df = build_selection_df_with_aliases(
    ecg_filenames=ecg_filenames,
    probs=probs,
    class_names=class_names,
    target_meta=TARGET_META,
    y_true=y_true,
    k_per_class=50,
    min_prob=0.85,
    max_duration_sec=20.0,
    duration_cache_path=str(ECG_DURATIONS_PATH)
)

sel_df.to_csv(ECG_PRED_PATH, index=False)
sel_df

[INFO] Estimating durations and keeping ECGs <= 20.0 s...
[WARN] duration cache has wrong length; recomputing.
[INFO] Saved duration cache to c:\UHull\ecg-xai\data\ecg_durations.npy
[INFO] Duration filter: keeping 11265/13187 ECGs (<= 20.0 s).
[CLASS 164889003 (atrial fibrillation)] picked 50 examples.
[CLASS 426783006 (sinus rhythm)] picked 50 examples.


,group_class,filename,sel_idx,duration_sec,prob_meta
0,164889003,C:\UHull\Data\Training_WFDB\A2249.mat,5347,16.968,0.940748
1,164889003,C:\UHull\Data\Training_WFDB\A2978.mat,6076,10.000,0.986492
2,164889003,C:\UHull\Data\Training_WFDB\A2618.mat,5716,14.474,0.988416
3,164889003,C:\UHull\Data\Training_WFDB\A0003.mat,3101,10.000,0.959976
4,164889003,C:\UHull\Data\Training_WFDB\A5377.mat,8475,19.122,0.990656
...,...,...,...,...,...
95,426783006,C:\UHull\Data\WFDB\HR00005.mat,8921,10.000,0.993680
96,426783006,C:\UHull\Data\WFDB\HR01101.mat,10017,10.000,0.992534
97,426783006,C:\UHull\Data\WFDB\HR02292.mat,11208,10.000,0.955642
98,426783006,C:\UHull\Data\WFDB\HR04184.mat,13100,10.000,0.993729


In [6]:
# ---- choose mode ----
run_mode = "eval"   # "eval" or "demo"

MODE_CFG = {
    "eval": {"max_examples_per_class": 50, "plot": False},
    "demo": {"max_examples_per_class": 3,  "plot": False},
}

In [7]:
cfg = MODE_CFG[run_mode]

OUT_BASE = ROOT / "outputs"
RUN_DIR = OUT_BASE / run_mode

# ---- cache check ----
def run_assets_exist(run_dir: Path) -> bool:
    return (
        (run_dir / "all_fused_payloads.joblib").exists()
        and (run_dir / "df_lime_all.parquet").exists()
        and (run_dir / "df_ts_all.parquet").exists()
        and (run_dir / "sel_df.parquet").exists()
    )

target_classes = list(TARGET_META.keys())

if run_assets_exist(RUN_DIR):
    print(f"[{run_mode}] Loading cached assets from: {RUN_DIR}")
    all_fused_payloads, df_lime_all, df_ts_all, sel_df_cached = load_run(RUN_DIR)
    
    # Optionally replace sel_df with cached one to keep indices consistent
    sel_df = sel_df_cached
else:
    print(f"[{run_mode}] No cache found. Running pipeline...\n")
    all_fused_payloads, df_lime_all, df_ts_all = run_fused_pipeline_for_classes(
        target_classes=target_classes,
        sel_df=sel_df,
        model=model,
        class_names=class_names,
        max_examples_per_class=cfg["max_examples_per_class"],
        plot=cfg["plot"],
    )

    save_run(
        RUN_DIR,
        all_fused_payloads,
        df_lime_all,
        df_ts_all,
        sel_df,
        meta={
            "mode": run_mode,
            "saved_at": datetime.now().isoformat(timespec="seconds"),
            "target_classes": target_classes,
            "max_examples_per_class": cfg["max_examples_per_class"],
            "plot": cfg["plot"],
        },
    )
    print(f"[{run_mode}] Saved to: {RUN_DIR}")

[eval] No cache found. Running pipeline...


=== [1/2] Processing class: 164889003 ===

 FUSED class=164889003 | window=0.5s | m_event=100 | m_feat=100 | topk_events=5
 LIME done: 50 rows in 13m07s
 TimeSHAP done: 50 rows in 10m17s
  Fusing: 50 common records


  Class 164889003 total: 23m25s
— Progress: 1/2 classes | ETA ~ 23m25s

=== [2/2] Processing class: 426783006 ===

 FUSED class=426783006 | window=0.25s | m_event=100 | m_feat=100 | topk_events=5


 LIME done: 50 rows in 12m30s
 TimeSHAP done: 50 rows in 9m56s
  Fusing: 50 common records


  Class 426783006 total: 22m27s
— Progress: 2/2 classes | ETA ~ 0m00s

 All classes complete in 45m53s


[eval] Saved to: c:\UHull\ecg-xai\outputs\eval


In [8]:
from eval import evaluate_all_payloads

df_eval_all = evaluate_all_payloads(
    all_payloads=all_fused_payloads,
    method_label="LIME+TimeSHAP",
    model=model,
    class_names=class_names,
)

df_eval_all

,meta_code,class_name,sel_idx,mat_path,method,strict_attauc,lenient_attauc,precision_k,strict_p_at_k,lenient_p_at_k,deletion_auc,faithfulness_gain,n_tokens
0,164889003,atrial fibrillation,1992,C:\UHull\Data\Training_2\Q0946.mat,LIME+TimeSHAP,0.878189,0.723214,20,0.50,1.0,0.293073,0.024195,168
1,164889003,atrial fibrillation,2212,C:\UHull\Data\Training_2\Q1179.mat,LIME+TimeSHAP,0.859948,0.635435,20,0.25,1.0,0.250375,0.105301,204
2,164889003,atrial fibrillation,2487,C:\UHull\Data\Training_2\Q1462.mat,LIME+TimeSHAP,0.844199,0.635585,20,0.30,0.7,0.294993,0.013402,372
3,164889003,atrial fibrillation,3101,C:\UHull\Data\Training_WFDB\A0003.mat,LIME+TimeSHAP,0.860280,0.650225,20,0.45,0.9,0.289826,-0.045046,300
4,164889003,atrial fibrillation,3124,C:\UHull\Data\Training_WFDB\A0026.mat,LIME+TimeSHAP,0.896007,0.739583,20,0.55,1.0,0.230502,0.204329,144
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,426783006,sinus rhythm,12529,C:\UHull\Data\WFDB\HR03613.mat,LIME+TimeSHAP,0.846094,1.000000,20,0.35,1.0,0.257836,0.014502,96
96,426783006,sinus rhythm,12542,C:\UHull\Data\WFDB\HR03626.mat,LIME+TimeSHAP,0.843778,1.000000,20,0.35,1.0,0.253848,0.039210,180
97,426783006,sinus rhythm,13100,C:\UHull\Data\WFDB\HR04184.mat,LIME+TimeSHAP,0.835764,1.000000,20,0.25,1.0,0.297479,-0.002890,144
98,426783006,sinus rhythm,13148,C:\UHull\Data\WFDB\HR04232.mat,LIME+TimeSHAP,0.856500,1.000000,20,0.35,1.0,0.289625,0.000269,120


In [12]:
df_eval_all.to_csv(ROOT / "data" / 'df_eval_attauc_deletion.csv')

### Stability to an Extra Heartbeat (All Target Classes)

We assess explanation stability under a synthetic perturbation where one existing
heartbeat is duplicated and re-inserted either at the **end** of the ECG or in
the **middle** of the recording.

For each target rhythm class in `TARGET_META`:

1. We take the example that was already explained and evaluated in `df_eval_all`.
2. We run `run_extra_beat_stability_experiment`, which:
   - creates two augmented versions of the ECG
     - one with an **extra beat appended at the end**
     - one with an **extra beat inserted in the middle**
   - recomputes **fused LIME + TimeSHAP** explanations for each version
   - compares region-level importance profiles over the *shared beats* using:
     - **Spearman rank correlation** (global ordering of important regions)
     - **Jaccard@K** overlap of the top-K most important regions.

This gives us per-class stability metrics for:

- original vs **extra-beat-at-end**
- original vs **extra-beat-in-middle**


In [10]:
from stability_eval import run_extra_beat_stability_experiment

MAX_PER_CLASS = 50  # None for all

stability_rows = []

df_eval_all = df_eval_all.copy()
df_eval_all["meta_code"] = df_eval_all["meta_code"].astype(str)

OUT_DIR = Path(ROOT / "outputs" / "extra_beat_aug")
OUT_DIR.mkdir(parents=True, exist_ok=True)

for meta_code_str, df_cls in df_eval_all.groupby("meta_code", sort=False):
    class_name = str(TARGET_META[str(meta_code_str)]["name"])

    # Optional: skip already-augmented mat paths
    df_cls = df_cls[~df_cls["mat_path"].str.contains(r"_extra_(end|mid)\.mat$", case=False, na=False)]

    if MAX_PER_CLASS is not None:
        df_cls = df_cls.head(MAX_PER_CLASS)

    print(f"\n=== Class {meta_code_str} ({class_name}) | n={len(df_cls)} ===")

    for _, row in df_cls.iterrows():
        mat_path = row["mat_path"]
        sel_idx  = int(row["sel_idx"])

        print(f"\n  -> [{meta_code_str}] sel_idx={sel_idx} | {mat_path}")

        metrics, sel_df_stab, fused_payloads_stab, df_lime_stab, df_ts_stab = run_extra_beat_stability_experiment(
            mat_path=mat_path,
            snomed_code=str(meta_code_str),
            model=model,
            class_names=class_names,
            augment_root=OUT_DIR,
        )

        # Helper to safely pull values (in case a metric is missing / NaN)
        end = metrics.get("extra_end", {})
        mid = metrics.get("extra_mid", {})

        stability_rows.append({
            "meta_code": str(meta_code_str),
            "class_name": class_name,
            "sel_idx": sel_idx,
            "mat_path": mat_path,

            "extra_end_spearman": end.get("spearman"),
            "extra_end_jaccard": end.get("jaccard_topk"),

            "extra_mid_spearman": mid.get("spearman"),
            "extra_mid_jaccard": mid.get("jaccard_topk"),

            # Optional extra fields if you added them:
            "extra_end_rbo": end.get("rbo"),
            "extra_end_wjacc": end.get("weighted_jaccard"),
            "extra_end_k_eff": end.get("k_eff"),

            "extra_mid_rbo": mid.get("rbo"),
            "extra_mid_wjacc": mid.get("weighted_jaccard"),
            "extra_mid_k_eff": mid.get("k_eff"),
        })

df_stability = pd.DataFrame(stability_rows)
df_stability


C:\Users\rajuvaneshwarn\AppData\Local\Temp\ipykernel_23124\1569309632.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_cls = df_cls[~df_cls["mat_path"].str.contains(r"_extra_(end|mid)\.mat$", case=False, na=False)]



=== Class 164889003 (atrial fibrillation) | n=50 ===

  -> [164889003] sel_idx=1992 | C:\UHull\Data\Training_2\Q0946.mat

=== [1/1] Processing class: 164889003 ===

 FUSED class=164889003 | window=0.5s | m_event=100 | m_feat=100 | topk_events=5
 LIME done: 3 rows in 0m45s
 TimeSHAP done: 3 rows in 0m38s
  Fusing: 3 common records


  Class 164889003 total: 1m24s
— Progress: 1/1 classes | ETA ~ 0m00s

 All classes complete in 1m24s

  -> [164889003] sel_idx=2212 | C:\UHull\Data\Training_2\Q1179.mat

=== [1/1] Processing class: 164889003 ===

 FUSED class=164889003 | window=0.5s | m_event=100 | m_feat=100 | topk_events=5


 LIME done: 3 rows in 0m52s
 TimeSHAP done: 3 rows in 0m40s
  Fusing: 3 common records


  Class 164889003 total: 1m32s
— Progress: 1/1 classes | ETA ~ 0m00s

 All classes complete in 1m32s

  -> [164889003] sel_idx=2487 | C:\UHull\Data\Training_2\Q1462.mat

=== [1/1] Processing class: 164889003 ===

 FUSED class=164889003 | window=0.5s | m_event=100 | m_feat=100 | topk_events=5


...

 LIME done: 3 rows in 1m30s
 TimeSHAP done: 3 rows in 1m08s
  Fusing: 3 common records


  Class 426783006 total: 2m39s
— Progress: 1/1 classes | ETA ~ 0m00s

 All classes complete in 2m39s

  -> [426783006] sel_idx=13174 | C:\UHull\Data\WFDB\HR04258.mat

=== [1/1] Processing class: 426783006 ===

 FUSED class=426783006 | window=0.25s | m_event=100 | m_feat=100 | topk_events=5


 LIME done: 3 rows in 1m32s
 TimeSHAP done: 3 rows in 1m12s
  Fusing: 3 common records


  Class 426783006 total: 2m45s
— Progress: 1/1 classes | ETA ~ 0m00s

 All classes complete in 2m45s


,meta_code,class_name,sel_idx,mat_path,extra_end_spearman,extra_end_jaccard,extra_mid_spearman,extra_mid_jaccard,extra_end_rbo,extra_end_wjacc,extra_end_k_eff,extra_mid_rbo,extra_mid_wjacc,extra_mid_k_eff
0,164889003,atrial fibrillation,1992,C:\UHull\Data\Training_2\Q0946.mat,1.0,1.0,0.945462,0.529412,0.833228,1.0,17.0,0.340433,0.941682,13.0
1,164889003,atrial fibrillation,2212,C:\UHull\Data\Training_2\Q1179.mat,1.0,1.0,0.974718,1.000000,0.717570,1.0,12.0,0.519036,0.970984,11.0
2,164889003,atrial fibrillation,2487,C:\UHull\Data\Training_2\Q1462.mat,1.0,1.0,0.916863,0.600000,0.878423,1.0,20.0,0.405152,0.954581,20.0
3,164889003,atrial fibrillation,3101,C:\UHull\Data\Training_WFDB\A0003.mat,1.0,1.0,0.887846,0.481481,0.878423,1.0,20.0,0.245542,0.929057,20.0
4,164889003,atrial fibrillation,3124,C:\UHull\Data\Training_WFDB\A0026.mat,1.0,1.0,0.912234,0.538462,0.745813,1.0,13.0,0.197972,0.929080,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,426783006,sinus rhythm,12529,C:\UHull\Data\WFDB\HR03613.mat,1.0,1.0,0.950845,1.000000,0.651322,1.0,10.0,0.285237,0.967647,10.0
96,426783006,sinus rhythm,12542,C:\UHull\Data\WFDB\HR03626.mat,1.0,1.0,0.928199,0.384615,0.849905,1.0,18.0,0.299687,0.950634,18.0
97,426783006,sinus rhythm,13100,C:\UHull\Data\WFDB\HR04184.mat,1.0,1.0,0.989635,0.666667,0.771232,1.0,14.0,0.555836,0.984753,10.0
98,426783006,sinus rhythm,13148,C:\UHull\Data\WFDB\HR04232.mat,1.0,1.0,0.932685,0.333333,0.717570,1.0,12.0,0.221334,0.927788,10.0


In [11]:
df_stability.to_csv(ROOT / "data" / 'df_eval_stability.csv')

### TBC

### Stability Summary Table

To make the results easier to inspect and report, we summarise the stability
metrics for each target class in a single table.


In [ ]:
rows = []

for code, info in stability_results.items():
    m = info["metrics"]
    m_end = m["extra_end"]
    m_mid = m["extra_mid"]

    rows.append({
        "SNOMED code": code,
        "Class name": info["name"],
        "Spearman (extra end)":  m_end["spearman"],
        "Jaccard@K (extra end)": m_end["jaccard_topk"],
        "Spearman (extra mid)":  m_mid["spearman"],
        "Jaccard@K (extra mid)": m_mid["jaccard_topk"],
    })

df_stability_summary = pd.DataFrame(rows)
df_stability_summary


Overall, the fused explanations were **perfectly stable** to appending an
extra heartbeat at the end of the recording across all classes
(Spearman ≈ 1.0, Jaccard@K = 1.0). This indicates that adding extra context
at the tail of the ECG does not affect the explanations for the existing beats.

In contrast, inserting an extra beat in the middle of the recording led to
a reduction in stability. Spearman rank correlation remained high
(≈ 0.94–0.97), suggesting that the global ordering of important regions is
largely preserved, but Jaccard@K dropped (e.g. 0.33–0.60), indicating that the
exact composition of the top-K most important regions is more sensitive to
perturbations in the internal temporal structure of the signal.


In [ ]:
import pandas as pd

# Group by SNOMED meta-code + human-readable name
summary = (
    df_eval_all
    .groupby(["meta_code", "class_name"])
    .agg(
        # --- AttAUC: ranking-based localisation accuracy ---
        strict_attauc_mean=("strict_attauc", "mean"),
        strict_attauc_std=("strict_attauc", "std"),
        lenient_attauc_mean=("lenient_attauc", "mean"),
        lenient_attauc_std=("lenient_attauc", "std"),

        # --- Deletion AUC: faithfulness (lower is better) ---
        deletion_auc_mean=("deletion_auc", "mean"),
        deletion_auc_std=("deletion_auc", "std"),

        # Token count (just descriptive)
        n_tokens_mean=("n_tokens", "mean"),
        n_tokens_std=("n_tokens", "std"),
    )
    .reset_index()
)

summary


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# For nicer x-axis labels
classes = summary["class_name"].values
x = np.arange(len(classes))
width = 0.35  # bar width

fig, ax = plt.subplots(figsize=(8, 4))

# Strict AttAUC: explainer vs strict, narrow definition of "correct"
ax.bar(
    x - width/2,
    summary["strict_attauc_mean"],
    width,
    yerr=summary["strict_attauc_std"],
    capsize=4,
    label="Strict AttAUC",
)

# Lenient AttAUC: explainer vs broader, more forgiving ground truth
ax.bar(
    x + width/2,
    summary["lenient_attauc_mean"],
    width,
    yerr=summary["lenient_attauc_std"],
    capsize=4,
    label="Lenient AttAUC",
)

ax.set_xticks(x)
ax.set_xticklabels(classes, rotation=20, ha="right")
ax.set_ylabel("AttAUC (ranking-based)")
ax.set_ylim(0, 1.05)
ax.set_title("Token-level localisation accuracy (AttAUC)")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

# Lower deletion AUC => more faithful (probability drops faster)
ax.bar(
    x,
    summary["deletion_auc_mean"],
    yerr=summary["deletion_auc_std"],
    capsize=4,
)

ax.set_xticks(x)
ax.set_xticklabels(classes, rotation=20, ha="right")
ax.set_ylabel("Deletion AUC (lower = more faithful)")
ax.set_title("Faithfulness of explanations via targeted deletion")
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

# Use strict AttAUC on x-axis and deletion AUC on y-axis
ax.scatter(
    summary["strict_attauc_mean"],
    summary["deletion_auc_mean"],
)

for _, row in summary.iterrows():
    ax.text(
        row["strict_attauc_mean"] + 0.005,
        row["deletion_auc_mean"] + 0.002,
        row["class_name"],
        fontsize=8,
    )

ax.set_xlabel("Strict AttAUC (higher = more accurate localisation)")
ax.set_ylabel("Deletion AUC (lower = more faithful)")
ax.set_title("Accuracy vs Faithfulness across diagnostic classes")
plt.tight_layout()
plt.show()
